In [4]:
import requests
import json
from bs4 import BeautifulSoup

In [10]:
#def get_movies(user):
    
#     """
#     This function is used to get the viewing date, movie URL, and user rating from user's diary on letterboxd
    
#     Parameter
#     ---------
#     user: string
#         your user on letterboxd
    
#     Returns
#     -------
#         a list of dictionary with a generated id, viewing date, movie URL, and rating
#     """
    
user = "gfac"
domain = "https://letterboxd.com/"
diary_url = domain + user + "/films/diary/"

r = requests.get(diary_url)
following_url = diary_url

data = []
movie_id = 1    # we need to generate an id to merge the DataFrames later

# while there is another page, the scrape will continue
while True:
    following_page = requests.get(following_url)
    soup = BeautifulSoup(following_page.content, "html.parser")

    table = soup.find("tbody")    # access the body
    rows = table.find_all("tr")    # access the rows

    # iterate over the rows of the table
    for row in rows:
        cols = row.find_all("td")    # the diary page is divided into columns
        movie_url = domain + cols[8].get("data-film-link")    # get the movie url
        date = cols[8].find("a")["data-viewing-date"]    # get the viewing date 
        user_rating = cols[8].find("a")["data-rating"]    # get the user rating
        rewatch = cols[8].find("a")["data-rewatch"]    # check if it's a rewatch
        data.append({
                "movie_id": movie_id,
                "movie_url": movie_url,
                "date": date,
                "user_rating": float(user_rating)/2,
                "rewatch?": rewatch
                    })
        movie_id += 1

    # check if there is another page of diary
    next_button = soup.find('a', class_='next')
    if next_button is None:
        break
    else:
        following_url = domain + next_button['href']

links = []
for d in data:
    links.append(d["movie_url"])
        
for u in links[:2]:

        r = requests.get(u)
        soup = BeautifulSoup(r.content, "html.parser")

        title = soup.select_one("#film-page-wrapper h1").get_text()
        year = soup.select_one("#featured-film-header small").get_text()
        directors = soup.select_one("#featured-film-header span").get_text()

        cast = [cast.text for cast in soup.find_all("a", {"class": "text-slug tooltip"}, limit=3)]

        genres = soup.find("div", {"class": "text-sluglist capitalize"})
        genres = [genres.text for genres in genres.find_all("a", {"class": "text-slug"})]

        table = soup.find("div", {"id": "tab-details"})

        rows_countries = table.find_all("div")[1]
        countries = [countries.text for countries in rows_countries.find_all("a", limit=2)]

        rows_languages = table.find_all("div")[2]
        language = [countries.text for countries in rows_languages.find_all("a", limit=2)]

        overall_rating = soup.find("meta", {"name": "twitter:data2"}).get("content")[:4]

        data.append({
            "data_id": data_id,
            "title": title,
            "year": year,
            "directors": directors,
            "cast": cast,
            "genres": genres,
            "countries": countries,
            "language": language,
            "overall_rating": float(overall_rating)})
        data_id += 1

[{'movie_id': 1,
  'movie_url': 'https://letterboxd.com//film/when-marnie-was-there/',
  'date': '2022-09-11',
  'user_rating': 4.0,
  'rewatch?': 'false'},
 {'movie_id': 2,
  'movie_url': 'https://letterboxd.com//film/the-fly-1986/',
  'date': '2022-09-11',
  'user_rating': 4.5,
  'rewatch?': 'false'},
 {'movie_id': 3,
  'movie_url': 'https://letterboxd.com//film/a-brighter-summer-day/',
  'date': '2022-09-05',
  'user_rating': 4.5,
  'rewatch?': 'false'},
 {'movie_id': 4,
  'movie_url': 'https://letterboxd.com//film/millennium-mambo/',
  'date': '2022-09-04',
  'user_rating': 4.0,
  'rewatch?': 'false'},
 {'movie_id': 5,
  'movie_url': 'https://letterboxd.com//film/nope/',
  'date': '2022-08-28',
  'user_rating': 4.0,
  'rewatch?': 'false'},
 {'movie_id': 6,
  'movie_url': 'https://letterboxd.com//film/prey-2022/',
  'date': '2022-08-21',
  'user_rating': 3.5,
  'rewatch?': 'false'},
 {'movie_id': 7,
  'movie_url': 'https://letterboxd.com//film/rear-window/',
  'date': '2022-08-07',


In [ ]:
# call the function get_movies and write it as a json file
diary_data = get_movies("gfac")
length = len(diary_data)

with open("diary_data.json", 'w') as f:
    json.dump(diary_data, f,  indent=2)

In [25]:
def get_info(url):

    data = []
    data_id = 1

    for u in url[:2]:

        r = requests.get(u)
        soup = BeautifulSoup(r.content, "html.parser")

        title = soup.select_one("#film-page-wrapper h1").get_text()
        year = soup.select_one("#featured-film-header small").get_text()
        directors = soup.select_one("#featured-film-header span").get_text()

        cast = [cast.text for cast in soup.find_all("a", {"class": "text-slug tooltip"}, limit=3)]

        genres = soup.find("div", {"class": "text-sluglist capitalize"})
        genres = [genres.text for genres in genres.find_all("a", {"class": "text-slug"})]

        table = soup.find("div", {"id": "tab-details"})

        rows_countries = table.find_all("div")[1]
        countries = [countries.text for countries in rows_countries.find_all("a", limit=2)]

        rows_languages = table.find_all("div")[2]
        language = [countries.text for countries in rows_languages.find_all("a", limit=2)]

        overall_rating = soup.find("meta", {"name": "twitter:data2"}).get("content")[:4]

        data.append({
            "data_id": data_id,
            "title": title,
            "year": year,
            "directors": directors,
            "cast": cast,
            "genres": genres,
            "countries": countries,
            "language": language,
            "overall_rating": float(overall_rating)})
        data_id += 1
        
    return data

In [26]:
# create a list of url
links = []
for d in diary_data:
    links.append(d["movie_url"])

# call the function get_info and write it as a json file
movie_data = get_info(links)

with open("movie_data.json", 'w') as f:
    json.dump(movie_data, f,  indent=2)

In [ ]:
import pandas as pd

a = pd.DataFrame(diary_data)
b = pd.DataFrame(data)

my_diary = a.merge(b, left_on="movie_id", right_on="data_id")
my_diary.drop(columns=["movie_url", "data_id"])
my_diary = my_diary[["date", "movie_id", "title", "year", "directors", "cast", "genres",
                     "countries", "language", "user_rating", "overall_rating", "watched_by", "liked_by"]]

my_diary.head()

In [ ]:
my_diary2 = pd.DataFrame(my_diary["cast"].to_list(), columns=["cast1", "cast2", "cast3"])
my_diary2.head()